# read data

In [1]:
import pandas as pd
import numpy as np

from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, classification_report
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,  Dense, Flatten, Dropout,Input

In [2]:
df= pd.read_csv('/Users/meghnaroy/Library/CloudStorage/OneDrive-purdue.edu/Research/MentalHealth/CHASE/CODE_New/IHOPE/Data/df2_new.csv')

In [3]:
final_feature_importances_df= pd.read_csv('/Users/meghnaroy/Library/CloudStorage/OneDrive-purdue.edu/Research/MentalHealth/CHASE/CODE_New/IHOPE/Data/DF_Scores_Imp.csv')


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, AdamW
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input

# Define the features and target
features = ['phone_score', 'sleep_score', 'leisure_score', 'social_score', 'me_score']
target = 'phq4_category'

In [5]:
uids = final_feature_importances_df['UID'].unique()

# kfold 

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Sequential, load_model


# Directory where all per‐fold and best models will be saved
model_dir = (
    'Saved_Models_1'
)
os.makedirs(model_dir, exist_ok=True)

results = []
uids = df['uid'].unique()

for uid in uids:
    df_uid = df[df['uid'] == uid]
    n_samples = len(df_uid)
    if n_samples < 160:
        print(f"Skipping UID {uid} ({n_samples} samples < 160)")
        continue

    X_all = df_uid[features].values
    y_all = df_uid[target].values


    y_all_encoded = to_categorical(y_all, num_classes=4)

    skf = KFold(5,shuffle=True, random_state=42)

    best_fold_acc = -np.inf
    best_fold_model_path = None

    fold_idx = 0
    for train_idx, val_idx in skf.split(X_all, y_all):
        fold_idx += 1

        X_train_raw = X_all[train_idx]
        X_val_raw   = X_all[val_idx]
        y_train     = y_all_encoded[train_idx]
        y_val       = y_all_encoded[val_idx]


        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train_raw)
        X_val   = scaler.transform(X_val_raw)


        model = Sequential([
            Input(shape=(X_train.shape[1],)),
            Dense(64, activation='relu'),
            Dense(64, activation='relu'),
            Dense(64, activation='relu'),
            Dense(4, activation='softmax')
        ])
        model.compile(
            optimizer=Adam(learning_rate=0.01),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )


        fold_model_path = os.path.join(
            model_dir,
            f'phq4_baseline1_uid_{uid}_fold{fold_idx}.keras'
        )
        checkpoint = ModelCheckpoint(
            filepath=fold_model_path,
            monitor='val_accuracy',
            mode='max',
            save_best_only=True,
            verbose=0
        )

        history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=50,
            batch_size=8,
            callbacks=[checkpoint],
            verbose=0
        )


        best_fold_model = load_model(fold_model_path)
        val_loss, val_acc = best_fold_model.evaluate(X_val, y_val, verbose=0)

        if val_acc > best_fold_acc:
            best_fold_acc = val_acc
            best_fold_model_path = fold_model_path


    if best_fold_model_path is not None:
        best_uid_model_path = os.path.join(
            model_dir,
            f'phq4_baseline1_uid_{uid}_best.keras'
        )
        # Overwrite if it already exists
        if os.path.exists(best_uid_model_path):
            os.remove(best_uid_model_path)
        os.replace(best_fold_model_path, best_uid_model_path)
        # Optionally, delete the other fold files to save space
        for f in os.listdir(model_dir):
            if (
                f.startswith(f'phq4_baseline1_uid_{uid}_fold') 
                and not f.endswith(f'_best.keras')
            ):
                try:
                    os.remove(os.path.join(model_dir, f))
                except OSError:
                    pass
    else:
        best_uid_model_path = None

    # 6) Store results for this UID
    results.append({
        'UID': uid,
        'Test_Accuracy': best_fold_acc,
        'Model_Path': best_uid_model_path
    })
    print(f"UID {uid} - Test accuracy: {best_fold_acc:.4f}")

results_df = pd.DataFrame(results)



UID 1ff6d7f34acb354430e7323a35ff7703 - Test accuracy: 0.8500
Skipping UID fa394f6d3d077bd5568fc3bc01580806 (157 samples < 160)
Skipping UID 596df3c263e97eb09d44a8535ec20ffa (35 samples < 160)
UID 3dad5f11680b432159b65838119ab87e - Test accuracy: 0.7500
UID a23be0935798553a76b4e74cfc1740f1 - Test accuracy: 1.0000
UID ac70fe1f8115ac361f2023269c011c3e - Test accuracy: 0.7826
UID 791c580346256bca5c697d19170a50b5 - Test accuracy: 1.0000
UID 2b3d267f18830c3c257b382edabef246 - Test accuracy: 0.9000
Skipping UID 46f1dfeda71a865074859c81bb314fd9 (37 samples < 160)
UID f4d488e3c8096842b645280d8f01ae43 - Test accuracy: 0.8800
UID 3569e2f520db9014b4acc4227a6421c1 - Test accuracy: 0.9630
UID 3bb377ba0acb7d8916010184df36aa57 - Test accuracy: 1.0000
UID 8391d8b451084771d0affcc26e34d773 - Test accuracy: 0.8182
UID 79d8490ab94fc7cd6530b1f22593d708 - Test accuracy: 0.9545
UID c31fd374e5d8227a31bf8f06f0402b96 - Test accuracy: 1.0000
UID 96cf40f9d3b2d69466161000cd69ae5c - Test accuracy: 0.9375
Skipping UI

In [10]:
results_df 

,UID,Test_Accuracy,Model_Path
0,1ff6d7f34acb354430e7323a35ff7703,0.850000,Saved_Models_1/phq4_baseline1_uid_1ff6d7f34acb...
1,3dad5f11680b432159b65838119ab87e,0.750000,Saved_Models_1/phq4_baseline1_uid_3dad5f11680b...
2,a23be0935798553a76b4e74cfc1740f1,1.000000,Saved_Models_1/phq4_baseline1_uid_a23be0935798...
3,ac70fe1f8115ac361f2023269c011c3e,0.782609,Saved_Models_1/phq4_baseline1_uid_ac70fe1f8115...
4,791c580346256bca5c697d19170a50b5,1.000000,Saved_Models_1/phq4_baseline1_uid_791c58034625...
...,...,...,...
116,97487f1632fbab90426f6760a0fe94ea,0.812500,Saved_Models_1/phq4_baseline1_uid_97487f1632fb...
117,5c446b55e04c641913c09fd39d01d32b,0.826087,Saved_Models_1/phq4_baseline1_uid_5c446b55e04c...
118,f5529fbae87a8d170937d3e39d5a63cc,0.812500,Saved_Models_1/phq4_baseline1_uid_f5529fbae87a...
119,03a0ce5623bfeb8aa3113605f7682215,0.944444,Saved_Models_1/phq4_baseline1_uid_03a0ce5623bf...


In [11]:
results_df['Test_Accuracy'].mean()

0.878253738249629